Incorporate this into the cleaning / making the crosswalk
Then overwrite the crosswalk

In [1]:
import boto3
import pandas as pd
import geopandas as gpd

s3 = boto3.client('s3')
bucket_name = "city-planning-entitlements"

In [3]:
file_name = "test_crosswalk_parcels_tracts"
s3.download_file(bucket_name, f'data/{file_name}.parquet', f'./data/{file_name}.parquet')
gdf = gpd.read_parquet(f'./data/{file_name}.parquet')

In [4]:
gdf.head()

,AIN,GEOID,geometry
0,2004001003,06037113232,POINT (6374037.180 1903103.568)
1,2004001004,06037113232,POINT (6374036.928 1903037.812)
2,2004001005,06037113232,POINT (6374034.289 1902971.779)
3,2004001008,06037113232,POINT (6373421.743 1903148.836)
4,2004001009,06037113232,POINT (6373317.694 1903145.445)


In [6]:
# Figure out which ones have duplicate geoms
gdf = gdf.assign(
    x = gdf.geometry.centroid.x, 
    y = gdf.geometry.centroid.y, 
)

In [7]:
duplicate_geom = (gdf.groupby(['x', 'y'])
                  .agg({'AIN':'count'})
                  .reset_index()
                  .rename(columns = {'AIN':'num_AIN'})
                 )

In [8]:
duplicate_geom.num_AIN.value_counts()

1      2115905
2         9257
3         2205
4         2030
6         1395
        ...   
207          1
205          1
203          1
197          1
614          1
Name: num_AIN, Length: 254, dtype: int64

In [ ]:
gdf2 = pd.merge(gdf, duplicate_geom, 
                on = ['x', 'y'], how = 'left', validate = 'm:1')

In [ ]:
# Redo parts of src/A3
# TOC parcels that planning sent over...it's likely that it's tagging one AIN, but not historical ones
# Will need to do a check across all the ones that are duplicates with same x, y
# Tag the TOC Tier it belongs to
# Our crosswalk_parcels_tracts should include a lot of AINs, 
# but still need to store num_AIN, as well as total_AIN (which counts AIN after duplicates are dropped)